In [1]:
import datetime
from dateutil.relativedelta import relativedelta
import calendar
EASTERN = ['PJMISO', 'MISO', 'ERCOT', 'SPPISO', 'NYISO', 'SPP']
WESTERN = ['WECC', 'CAISO']
PEAK = ['onpeak', 'offpeak', 'flat', '2x16H', '7x8']
NUM_HOLIDAYS = {1:1, 2:0, 3:0, 4:0, 5:1, 6:0, 7:1, 8:0, 9:1, 10:0, 11:1, 12:1}
QUARTER = {1:[1, 2, 3], 2:[4, 5, 6], 3:[7, 8, 9], 4:[10, 11, 12]}

In [2]:
# Find number of Mon/Tue/.../Sun between two dates
def num_days_bw(s, e, wd):
    num_weeks, re = divmod((e - s).days, 7)
    if (wd - s.isoweekday() ) % 7 <= re:
        return num_weeks + 1
    else:
        return num_weeks

In [3]:
def get_hours(iso, peak_type, period):
    
    # Check ISO
    if iso not in EASTERN and iso not in WESTERN:
        print('Please enter correct ISO type.')
        return

    # Check peak type
    if peak_type not in PEAK:
        print('Please enter correct peak type, one of onpeak/offpeak/flat/2x16H/7x8.')
        return
    
    # If annual
    if period[-1] == 'A':
        year = int(period[:-1])
        start_date = datetime.date(year, 1, 1)
        end_date = datetime.date(year, 12, 31)
        num_holidays = 6
        # If these holidays are on Saturdays we have one less NERC holiday on work days
        # Only affect Eastern ISOs
        if iso in EASTERN:
            if datetime.date(year, 1, 1).isoweekday() == 6:
                num_holidays -= 1
            if datetime.date(year, 7, 4).isoweekday() == 6:
                num_holidays -= 1
            if datetime.date(year, 12, 25).isoweekday() == 6:
                num_holidays -= 1
            
    # If quarter
    elif period[-2] == 'Q':
        q = int(period[-1])
        year = int(period[:-2])
        if q == 1 or q == 4:
            start_date = datetime.date(year, QUARTER[q][0], 1)
            end_date = datetime.date(year, QUARTER[q][-1], 31)
        elif q == 2 or q == 3:
            start_date = datetime.date(year, QUARTER[q][0], 1)
            end_date = datetime.date(year, QUARTER[q][-1], 30)
        else:
            print('Please enter a valid quarter number.')
            return
        num_holidays = 0
        for i in QUARTER[q]:
            num_holidays += NUM_HOLIDAYS[i]
        if iso in EASTERN:
            if q == 1 and datetime.date(year, 1, 1).isoweekday() == 6:
                num_holidays -= 1
            elif q == 3 and datetime.date(year, 7, 4).isoweekday() == 6:
                num_holidays -= 1
            elif q == 4 and datetime.date(year, 12, 25).isoweekday() == 6:
                num_holidays -= 1
            
    # If month or date
    else:
        # If month
        try:
            start_date = datetime.datetime.strptime(period, '%Y%b').date()
            end_date = start_date + relativedelta(months = +1) + relativedelta(days = -1)
            year = start_date.year
            m = start_date.month
            num_holidays = NUM_HOLIDAYS[m]
            if iso in EASTERN:
                if m == 1 and datetime.date(year, 1, 1).isoweekday() == 6:
                    num_holidays -= 1
                elif m == 7 and datetime.date(year, 7, 4).isoweekday() == 6:
                    num_holidays -= 1
                elif m == 12 and datetime.date(year, 12, 25).isoweekday() == 6:
                    num_holidays -= 1
        except ValueError:
            # If date
            try:
                start_date = datetime.datetime.strptime(period, '%Y-%m-%d').date()
                end_date = start_date 
                year = start_date.year
                m = start_date.month
            except ValueError:
                print('Please enter a valid period.')
                return
    
    # Calculate number of work days and weekends/holidays in a given period
    num_days = (end_date - start_date).days + 1                         
    # Not a single day
    if num_days != 1:
        num_sat = num_days_bw(start_date, end_date, 6)
        num_sun = num_days_bw(start_date, end_date, 7)
        if iso in EASTERN:
            num_wes = num_sat + num_sun
        else:
            num_wes = num_sun
        num_wes += num_holidays
        num_wds = num_days - num_wes
    # Single day
    else:
        if start_date.isoweekday() == 7:
            num_wes = 1
        elif start_date.isoweekday() == 6 and iso in EASTERN:
            num_wes = 1
        elif datetime.date(year, 1, 1).isoweekday() != 7 and start_date == datetime.date(year, 1, 1):
            num_wes = 1
        elif datetime.date(year, 7, 4).isoweekday() != 7 and start_date == datetime.date(year, 7, 4):
            num_wes = 1
        elif datetime.date(year, 12, 25).isoweekday() != 7 and start_date == datetime.date(year, 12, 25):
            num_wes = 1
        elif datetime.date(year, 1, 1).isoweekday() == 7 and start_date == datetime.date(year, 1, 2):
            num_wes = 1
        elif datetime.date(year, 7, 4).isoweekday() == 7 and start_date == datetime.date(year, 7, 5):
            num_wes = 1
        elif datetime.date(year, 12, 25).isoweekday() == 7 and start_date == datetime.date(year, 12, 26):
            num_wes = 1
        elif (start_date.month == 5 and start_date.isoweekday() == 1 
              and (start_date + relativedelta(weeks=+1)).month == 6):
            num_wes = 1
        elif (start_date.month == 9 and start_date.isoweekday() == 1 
              and (start_date + relativedelta(weeks=-1)).month == 8):
            num_wes = 1
        elif (start_date.month == 11 and start_date.isoweekday() == 4 
              and (start_date + relativedelta(weeks=-4)).month == 10
              and (start_date + relativedelta(weeks=-3)).month == 11):
            num_wes = 1
        else:
            num_wes = 0
        num_wds = 1 - num_wes
    
    # Calculate number of hours
    if peak_type == 'flat':
        num_hour = 24 * num_days   
    elif peak_type == 'onpeak':
        num_hour = 16 * num_wds
    elif peak_type == 'offpeak':
        num_hour = 24 * num_days - 16 * num_wds
    elif peak_type == '2x16H':
        num_hour = 16 * num_wes
    elif peak_type == '7x8':
        num_hour = 8 * num_days
    
    # Adjust for Daylight Saving
    if iso != 'MISO':
        if period[-2] == 'Q':
            q = int(period[-1])
            if q == 1:
                if peak_type in ['flat', 'offpeak', '7x8']:
                    num_hour -= 1
            elif q == 4:
                if peak_type in ['flat', 'offpeak', '7x8']:
                    num_hour += 1
        else:
            if num_days != 1:
                m = start_date.month
                if m == 3:
                    if peak_type in ['flat', 'offpeak', '7x8']:
                        num_hour -= 1
                elif m == 11:
                    if peak_type in ['flat', 'offpeak', '7x8']:
                        num_hour += 1
            else:
                if (start_date.month == 3 and start_date.isoweekday() == 7 
                    and (start_date + relativedelta(weeks=-1)).month == 3
                    and (start_date + relativedelta(weeks=-2)).month == 2):
                    if peak_type in ['flat', 'offpeak', '7x8']:
                        num_hour -= 1
                elif (start_date.month == 11 and start_date.isoweekday() == 7 
                      and (start_date + relativedelta(weeks=-1)).month == 10):
                    if peak_type in ['flat', 'offpeak', '7x8']:
                        num_hour += 1
    
    return [iso, peak_type, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), num_hour]
    

In [4]:
print(get_hours("CAISO", "offpeak", "2020Jul"))

['CAISO', 'offpeak', '2020-07-01', '2020-07-31', 328]
